## Listen to example wav from GuitarSet dataset

In [ ]:
import IPython
IPython.display.Audio("guitarset/test/audio/00_BN1-129-Eb_comp_hex.wav")

In [ ]:
import fluidsynth
from midi2audio import FluidSynth
fs = FluidSynth()
fs.midi_to_audio("guitarset/test/labels/00_BN1-129-Eb_comp.jams.mid", 'output.wav')
IPython.display.Audio("output.wav")

In [ ]:
import os
import pprint
pp = pprint.PrettyPrinter(indent=4)

def get_all_midis(directory):
    listOfFiles=[]
    for (dirpath, dirnames, filenames) in os.walk(directory):
        for file in filenames:
            if file[-3:] == "mid":
                listOfFiles.append(os.path.join(dirpath, file))
    return listOfFiles

midis_filenames = get_all_midis("guitarset/test")
example_midi_file = midis_filenames[0]

fs.midi_to_audio(example_midi_file, 'output.wav')
IPython.display.Audio("output.wav")




In [ ]:
import pretty_midi
example_midi = pretty_midi.PrettyMIDI(example_midi_file)
list_of_instruments = [pretty_midi.program_to_instrument_name(program_int) for program_int in range(0,126)]
print("List of instruments:")
pp.pprint(list_of_instruments)
print("Instruments of example:")
pp.pprint([pretty_midi.program_to_instrument_name(instrument.program) for instrument in example_midi.instruments])

### Convert  instrument of example

In [ ]:

def convert_to_another_instrument(midi, program=1):
    new_instruments_list = []
    for instrument in midi.instruments:
        new_instruments_list.append(pretty_midi.Instrument(program=program))
        for note in instrument.notes:
            new_instruments_list[-1].notes.append(note)
    midi.instruments = new_instruments_list
    return midi


In [ ]:
flute_program = pretty_midi.instrument_name_to_program('Flute')
example_midi = convert_to_another_instrument(example_midi, flute_program)
example_midi.write("flute_conversion.mid")
fs.midi_to_audio("flute_conversion.mid", 'flute_conversion.wav')
IPython.display.Audio("flute_conversion.wav")

In [ ]:
piano_program = pretty_midi.instrument_name_to_program('Acoustic Grand Piano')
example_midi = convert_to_another_instrument(example_midi, piano_program)
example_midi.write("piano_conversion.mid")
fs.midi_to_audio("piano_conversion.mid", 'piano_conversion.wav')
IPython.display.Audio("piano_conversion.wav")

## Let's create some datasets for different instruments!

In [ ]:
instrument_list = [
                    'Bright Acoustic Piano',
                    'Electric Grand Piano',
                     'Honky-tonk Piano',
                     'Electric Piano 1',
                     'Electric Piano 2',
#                     'Harpsichord',
#                     'Clavinet',
#                     'Celesta',
#                     'Glockenspiel',
#                     'Music Box',
                     'Vibraphone',
#                     'Marimba',
                     'Xylophone',
#                     'Tubular Bells',
#                     'Dulcimer',
#                     'Drawbar Organ',
#                     'Percussive Organ',
                     'Rock Organ',
                     'Church Organ',
#                     'Reed Organ',
                     'Accordion',
                     'Harmonica',
#                     'Tango Accordion',
                    'Acoustic Guitar (nylon)',
                     'Acoustic Guitar (steel)',
                     'Electric Guitar (jazz)',
                     'Electric Guitar (clean)',
                     'Electric Guitar (muted)',
                     'Overdriven Guitar',
                     'Distortion Guitar',
                     'Guitar Harmonics',
                     'Acoustic Bass',
                     'Electric Bass (finger)',
                     'Electric Bass (pick)',
                     'Fretless Bass',
                     'Slap Bass 1',
                     'Slap Bass 2',
                     'Synth Bass 1',
                     'Synth Bass 2',
                     'Violin',
                     'Viola',
                    'Cello',
                     'Contrabass',
#                     'Tremolo Strings',
#                     'Pizzicato Strings',
                     'Orchestral Harp',
#                     'Timpani',
                     'String Ensemble 1',
#                     'String Ensemble 2',
                     'Synth Strings 1',
#                     'Synth Strings 2',
#                     'Choir Aahs',
#                     'Voice Oohs',
#                     'Synth Choir',
#                     'Orchestra Hit',
                     'Trumpet',
#                     'Trombone',
                     'Tuba',
#                     'Muted Trumpet',
#                     'French Horn',
#                     'Brass Section',
#                     'Synth Brass 1',
#                     'Synth Brass 2',
#                     'Soprano Sax',
#                     'Alto Sax',
#                     'Tenor Sax',
#                     'Baritone Sax',
#                     'Oboe',
#                     'English Horn',
#                     'Bassoon',
#                     'Clarinet',
#                     'Piccolo',
#                     'Flute',
#                     'Recorder',
#                     'Pan Flute',
#                     'Blown bottle',
#                     'Shakuhachi',
#                     'Whistle',
#                    'Ocarina',
                    'Acoustic Grand Piano',]
print("Generating datasets for following instruments:")
print(instrument_list)

In [ ]:
%%capture
import shutil
import os, sys
from IPython.utils import io
from sklearn.model_selection import train_test_split


def standarize_name(name):
    return name.lower().replace(' ', '_').replace('(','').replace(')','')

def make_standard_directories(path):
    print(f"Creating directory {path} test/val/train...")
    os.makedirs(f'{path}')
    os.makedirs(f'{path}/test')
    os.makedirs(f'{path}/val')
    os.makedirs(f'{path}/train')

def copy_labels_to_target_dir(label_list, target_dir):
    print(f"Copying labels to {target_dir}/labels...")
    os.makedirs(f'{target_dir}/labels')
    for label in label_list:
        shutil.copy(label, target_dir+"/labels/")

def synthesize_instrument_for_labels(labels_directory, output_directory, instrument_name):
    label_files = get_all_midis(labels_directory)
    program = pretty_midi.instrument_name_to_program(instrument_name)
    print(f"Detected program for given instrument {instrument_name}: {program}")
    os.makedirs(f'{output_directory}')
    for label in label_files:
            midi = pretty_midi.PrettyMIDI(label)
            midi = convert_to_another_instrument(midi, program)
            midi.write(f'{output_directory}/{os.path.basename(label)}.mid')
            with io.capture_output() as captured:
                fs.midi_to_audio(f'{output_directory}/{os.path.basename(label)}.mid',
                                 f'{output_directory}/{os.path.basename(label)}.wav')
            os.remove(f'{output_directory}/{os.path.basename(label)}.mid')
            os.rename(f'{output_directory}/{os.path.basename(label)}.wav',
                      f'{output_directory}/{os.path.basename(label)[:-9]}_hex.wav')

def create_files_for_phase(labels, instrument, phase):
    directory_name = f'synthesize_{standarize_name(instrument)}'
    os.makedirs(f'{directory_name}/{phase}')
    copy_labels_to_target_dir(labels, f'{directory_name}/{phase}')
    synthesize_instrument_for_labels(f'{directory_name}/{phase}/',
                                     f'{directory_name}/{phase}/audio',
                                     instrument)

def generate_datasets_for_instruments(instrument):
    train_filenames = get_all_midis("guitarset/train")
    test_filenames = get_all_midis("guitarset/test")
    val_filenames = get_all_midis("guitarset/val")
    directory_name = f'synthesize_{standarize_name(instrument)}'
    os.makedirs(directory_name)
    create_files_for_phase(train_filenames, instrument, 'train')
    create_files_for_phase(test_filenames, instrument, 'test')
    create_files_for_phase(val_filenames, instrument, 'val')

#generate_datasets_for_instruments(instrument_list)


In [ ]:
from multiprocessing import Pool

with Pool(12) as pool:
    pool.map(generate_datasets_for_instruments, instrument_list)